#### Align reads using bowtie. This command will output a sam file (alignment file) and a .log file which contains the details on how the alignment went. (I need gunzip, but probably need to update bowtie)

In [2]:
!gunzip -c netseq_data/clean_Trimmed_wt-mmc.fastq.gz | bowtie -S -m 1 -v 0 indexes/NC_000913.2 '-' > wt_mmc_NET.sam 2> wt_mmc_NET.log 

#### Convert sam to bam and only keep reads that aligned, convert bam to bed:

In [ ]:
!samtools view -b -h -F 4 wt_mmc_NET.sam > wt_mmc_NET_aligned.bam
!bedtools bamtobed -i wt_mmc_NET_aligned.bam > wt_mmc_NET_aligned.bed 

#### Edit befiles with pandas to only include the coordinate of the 3' base (needs to be double checked)

In [ ]:
import pandas as pd

In [ ]:
reads=pd.read_table('wt_mmc_NET_aligned.bed',header=None)
plus_reads=reads[reads[5]=='+']
minus_reads=reads[reads[5]=='-']
plus_reads[2]=plus_reads.loc[:,1].apply(lambda x:x+1)
plus_reads.to_csv('plus_3end.bed',header=False,sep='\t',index=False)
minus_reads[1]=minus_reads.loc[:,2].apply(lambda x:x-1)
minus_reads.to_csv('minus_3end.bed',header=False,sep='\t',index=False)

#### Use updated bedifles that were just made to determine the amount of 3'end for every position in each gene

In [ ]:
bedtools coverage -d -a NC_000913.2.gtf -b plus_3end.bed > wt_mmc_NET_plus_genecov.bed
bedtools coverage -d -a NC_000913.2.gtf -b minus_3end.bed > wt_mmc_NET_minus_genecov.bed